<a href="https://colab.research.google.com/github/Hisernberg/ai-agent-projects/blob/main/Copy_of_context_awareness_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install  chromadb pypdf pytesseract pdf2image sentence-transformers
!pip install langchain faiss-cpu PyMuPDF requests langchain-community
!apt-get install poppler-utils
!apt install tesseract-ocr
!curl -fsSL https://ollama.com/install.sh | sh
!nohup ollama serve > output.log 2>&1 &
!ollama pull phi4

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 5.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 80.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.4/2.4 MB 74.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.3/302.3 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 20.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 72.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 81.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.9/55.9 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.1/89.1 kB 6.1 MB/s eta 0:00:

In [ ]:
import os
from langchain.document_loaders import PyPDFLoader
from langchain.schema import Document
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import Chroma
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.llms import Ollama
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from pdf2image import convert_from_path
import pytesseract
from PIL import Image
import requests


In [ ]:
pdf_url = "https://physics.bme.hu/sites/physics.bme.hu/files/users/BMETE15MF74_kov/Backpropagation%20Step%20by%20Step.pdf"  # Replace with your preferred PDF URL
pdf_path = "sample.pdf"  # Replace with your file


# Download the PDF if it doesn't exist
if not os.path.exists(pdf_path):
    print(f"Downloading PDF from {pdf_url}...")
    response = requests.get(pdf_url)
    response.raise_for_status()  # Raise an exception for bad responses
    with open(pdf_path, "wb") as f:
        f.write(response.content)
    print(f"PDF saved to {pdf_path}")


text_loader = PyPDFLoader(pdf_path)
text_docs = text_loader.load()

# Load text-based pages
text_loader = PyPDFLoader(pdf_path)
text_docs = text_loader.load()

# Convert all pages to images for OCR
print("🔍 Running OCR on image-based pages...")
image_pages = convert_from_path(pdf_path)
ocr_docs = []

for i, image in enumerate(image_pages):
    text = pytesseract.image_to_string(image)
    if text.strip():
        ocr_docs.append(Document(page_content=text))

# Combine text + OCR docs
all_docs = text_docs + ocr_docs

# ==== Step 2: Split and Embed ====
splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_documents(all_docs)

embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
persist_dir = "./ollama_pdf_memory"

if os.path.exists(persist_dir):
    print("📁 Loading existing vector memory")
    vectordb = Chroma(persist_directory=persist_dir, embedding_function=embedding)
else:
    print("💾 Creating vector store...")
    vectordb = Chroma.from_documents(chunks, embedding=embedding, persist_directory=persist_dir)
    vectordb.persist()

# ==== Step 3: Set up Ollama and Memory ====
llm = Ollama(model="phi4")  # Or mistral, gemma, etc.

memory = ConversationBufferMemory(
    memory_key="chat_history",
    return_messages=True
)

qa_chain = ConversationalRetrievalChain.from_llm(
    llm=llm,
    retriever=vectordb.as_retriever(),
    memory=memory
)

# ==== Step 4: Chat Loop ====
print("\n🤖 PDF Chatbot (with Text + Image Support) — Type 'exit' to quit\n")
while True:
    query = input("You: ")
    if query.lower() in ["exit", "quit"]:
        print("👋 Bye!")
        break
    response = qa_chain.run({"question": query})
    print("Bot:", response)


PDF saved to sample.pdf
🔍 Running OCR on image-based pages...


<ipython-input-3-62842641991b>:39: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embedding = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.war

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

...

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

💾 Creating vector store...


<ipython-input-3-62842641991b>:48: LangChainDeprecationWarning: Since Chroma 0.4.x the manual persistence method is no longer supported as docs are automatically persisted.
  vectordb.persist()
<ipython-input-3-62842641991b>:51: LangChainDeprecationWarning: The class `Ollama` was deprecated in LangChain 0.3.1 and will be removed in 1.0.0. An updated version of the class exists in the :class:`~langchain-ollama package and should be used instead. To use it run `pip install -U :class:`~langchain-ollama` and import as `from :class:`~langchain_ollama import OllamaLLM``.
  llm = Ollama(model="phi4")  # Or mistral, gemma, etc.
<ipython-input-3-62842641991b>:53: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory(



🤖 PDF Chatbot (with Text + Image Support) — Type 'exit' to quit

You: quit
👋 Bye!


In [ ]:
test_questions = [
    "What is the purpose of backpropagation in neural networks?",
    "Explain the difference between forward and backward pass.",
    "What is the derivative of the sigmoid function used in backpropagation?",
    "In the document’s example, what are the initial weights and biases of the network?",
    "How is the error term δ (delta) calculated for the hidden layer?",
    "In Step 3 of the backpropagation example, how is the output of neuron o1 calculated?",
    "How is the total error (E_total) computed for the output layer?",
    "What are the updated weights after one iteration of training in the example?",
    "Why is it important to compute the gradient with respect to each weight?",
    "What role does the learning rate play in backpropagation?",
    "Can backpropagation be used with ReLU activation? Why or why not?",
    "Does backpropagation work for unsupervised learning tasks?"
]

print("\n🧪 Running Backpropagation Test Suite...\n")

for i, question in enumerate(test_questions, 1):
    print(f"Q{i}: {question}")
    try:
        answer = qa_chain.run({"question": question})
        print("🧠 Bot:", answer.strip())
    except Exception as e:
        print("⚠️ Error:", str(e))
    print("-" * 80)


🧪 Running Backpropagation Test Suite...

Q1: What is the purpose of backpropagation in neural networks?


<ipython-input-4-3ad7c85771b8>:21: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  answer = qa_chain.run({"question": question})


🧠 Bot: The purpose of backpropagation in neural networks is to update the weights of the network in order to minimize prediction error. This process involves adjusting the weights so that the network can more accurately map arbitrary inputs to outputs. Backpropagation works by calculating the gradient of the loss function with respect to each weight and using this information to make incremental adjustments to the weights. This iterative process helps improve the model's performance over time.
--------------------------------------------------------------------------------
Q2: Explain the difference between forward and backward pass.
🧠 Bot: In neural networks, the processes of the forward pass and backward pass are crucial for training models effectively.

1. **Forward Pass**: 
   - The forward pass involves computing the output of the network given a set of inputs.
   - During this phase, input data is fed through the layers of the network, with each layer performing computations base